In [8]:
import requests
import json
import dotenv
import os

dotenv.load_dotenv()

OPENROUTER_API_KEY = os.environ["OPENROUTER_KEY"]
def get_response(message):
    response = requests.post(
    url="https://openrouter.ai/api/v1/chat/completions",
    headers={
        "Authorization": f"Bearer {OPENROUTER_API_KEY}",
    },
    data=json.dumps({
        "model": "mistralai/mixtral-8x22b", # Optional
        "messages": [
        {"role": "user", "content": message}
        ]
    })
    ).json()
    
    return response


response= get_response("1+ 3+ 5 = ")

In [9]:
print(response['choices'][0]['message']['content'])


11




In [4]:
import json
with open("gpt-crawler/output-1.json", "r") as f:
    data = json.load(f)

In [9]:
content_map = {}
titles = []
for topic in data:
    titles.append(topic['title'])
    content_map[topic['title']] = topic['html']
        

In [25]:
def templater(title, content):
    return f"""
    Given the following documentation in HTML, please first summarize step by step what the section is about and then provide a code snippet that demonstrates the functionality of the section.
    ################################################
    You are given the title: {title}
    ################################################
    Here is the documentation:
    {content}
    """
    


def ex_templater(summary):
    return f"""
    Given the following summary, please write a coding exercise for the user to complete. You should write most of the code out already, and only leave a few lines blank for the user to write out
    
    You should only generate the instructions and the code exercise. The solutions and explanations are to be omitted.
    
    The following example is an example of what you should output:
    ################################################
    Example 1:
    Write code to assign the value of 1+2 to the variable a, and print it.
    
    a = 1+2
    ###TODO: print out the value of a (roughly 1 line)
    ________________________________________________
    Example 2:
    Write code to assign the value of 3+4 to the variable b, and print it.
    
    b = 3+4
    ###TODO: print out the value of b (roughly 1 line)
    ################################################
    Here is the summary:
    {summary}
    """


for i in titles[:10]:
    topic_ex = content_map[i]
    template = templater(i, topic_ex)
    res = get_response(template)
    
    summary = res['choices'][0]['message']['content']
    ex_template = ex_templater(summary)
    res = get_response(ex_template)
    print(res['choices'][0]['message']['content'])
    
    break
    
    

Here's a coding exercise based on the summary:

Compute the 1D discrete Fourier transform of the given input tensor and print the result.

```python
import torch

# Create a sample input tensor
x = torch.tensor([1, 2, 3, 4, 5, 6, 7, 8], dtype=torch.float32)

# TODO: Compute the 1D discrete Fourier transform of x (roughly 1 line)

print("FFT output:", y)
```


In [26]:
def answer_template(exercise):
    return f"""
    Given the following coding exercise, please write out the code that would complete the exercise. You should write out the code that would complete the exercise.
    
    Please also generate test cases for the code that you write. You should ensure that the testcases work.
    
    {exercise}

"""
temp_res = res['choices'][0]['message']['content']

ans_temp = answer_template(res['choices'][0]['message']['content'])
res = get_response(ans_temp)
print(res['choices'][0]['message']['content'])

Certainly! Here's the completed code for the given coding exercise, along with test cases:

```python
def is_palindrome(s):
    # Remove non-alphanumeric characters and convert to lowercase
    s = ''.join(c.lower() for c in s if c.isalnum())
    
    # Check if the string is equal to its reverse
    return s == s[::-1]
```

Test Cases:

```python
def test_is_palindrome():
    # Test case 1: Empty string
    assert is_palindrome("") == True
    
    # Test case 2: Single character
    assert is_palindrome("a") == True
    
    # Test case 3: Simple palindrome
    assert is_palindrome("racecar") == True
    
    # Test case 4: Palindrome with spaces and punctuation
    assert is_palindrome("A man, a plan, a canal: Panama") == True
    
    # Test case 5: Non-palindrome
    assert is_palindrome("hello") == False
    
    # Test case 6: Palindrome with mixed case
    assert is_palindrome("Madam") == True
    
    # Test case 7: Palindrome with numbers
    assert is_palindrome("12321") == 